<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/PGAsampling_FFEmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

In [2]:
%%time 
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes

CPU times: user 711 ms, sys: 107 ms, total: 819 ms
Wall time: 1min 35s


In [0]:
import glob
from math import sqrt
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import math

%matplotlib inline

# Mount drive


---



In [4]:
root = drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Path to input


---



In [0]:
path = '/content/drive/My Drive/Colab Notebooks/FFE/data/input'
pathPGAs = '/content/drive/My Drive/Colab Notebooks/FFE/data/input/PGAs'

path_output = '/content/drive/My Drive/Colab Notebooks/FFE/data/input/Ignition_prob_from_PGAs'

In [23]:
print(os.listdir(path))
print(os.listdir(pathPGAs))

['buildings_raw.cpg', 'buildings_raw.prj', 'buildings_raw.shp', 'buildings_raw.shx', 'buildings_raw.qpj', 'buildings_raw.dbf', 'GD_wind.csv', 'SA2.qpj', 'SA2.prj', 'SA2.dbf', 'SA2.shp', 'SA2.cpg', 'SA2.shx', 'PGAs', 'Ignition_prob_from_PGAs']
['WairarapNich.prj', 'WairarapNich.dbf', 'WairarapNich.cpg', 'WairarapNich.shx', 'WairarapNich.qpj', 'WairarapNich.shp', 'WellWHV.prj', 'WellWHV.shp', 'WellWHV.dbf', 'WellWHV.qpj', 'WellWHV.shx', 'Wairau.qpj', 'Wairau.dbf', 'Wairau.shx', 'Wairau.prj', 'Wairau.shp', 'HikWgtmin.shp', 'HikWgtmin.dbf', 'HikWgtmin.qpj', 'HikWgtmin.prj', 'HikWgtmin.shx', 'HikWgtmax.qpj', 'HikWgtmax.shx', 'HikWgtmax.shp', 'HikWgtmax.prj', 'HikWgtmax.dbf']


# Load section from exisiting functions


---



In [0]:
# import functions
# !git clone https://github.com/37stu37/FFE/.git
# %load /content/drive/My Drive/Colab Notebooks/FFE/sampling_PGA.ipynb

In [0]:
def Ignition_Probability(SA_data, PGA_data, percentile, building_data):
  data = SA_data.merge(PGA_data, how='left', left_on='SA2_ID', right_on='SA2')
  data['SA_IgnPrb'] = 1/(np.exp(-1*(-6.755+8.463+data[percentile]+0.0000984+data['PopDensity']+0.001523+data['BldgFloo_1'])))
  data['IgnProb_bl'] = 1 - np.power(1-data['SA_IgnPrb'], +1/data['NumberBldg'])
  data.drop('geometry_y', axis=1, inplace=True)
  data.rename(columns={'geometry_x': 'geometry'}, inplace=True)
  crs = {'init': 'EPSG:2193'}
  data_gdf = gpd.GeoDataFrame(data)
  buildings_with_ignition = gpd.sjoin(building_data, data_gdf, how='inner', op='within')
  buildings_with_ignition = buildings_with_ignition[['TARGET_FID', 'suburb_loc', 'Combustibl', 'SHAPE_Leng', 'SHAPE_Area',
       'AU2013Num', 'IgnProb_bl_left', 'RandProb', 'geometry']]
  buildings_with_ignition.rename(columns={'IgnProb_bl_left': 'IgnProb_bl'}, inplace=True)
  return buildings_with_ignition

# Iterate through PGA files and create PGA output fils


---



In [0]:
# SA2 input data
SA2 = gpd.read_file(os.path.join(path, "SA2.shp"))
# buildings data
buildings = gpd.read_file(os.path.join(path, "buildings_raw.shp"))
# PGA files
percentiles = ['pone', 'ptwo', 'pthree', 'pfour', 'pfive', 'psix', 'pseven']
for (root,dirs,files) in os.walk(os.path.join(pathPGAs)): 
    print (root) 
    # print (dirs) 
    print (files) 
    for file in files:        
        if file.endswith(".shp"):  
            print(file)
            for percentile in percentiles:
                print (percentile)
                PGA = gpd.read_file(os.path.join(pathPGAs, file))
                ign_map = Ignition_Probability(SA2, PGA, percentile, buildings)
                ign_map.to_file(os.path.join(path_output, "buildings_"+str(file)+"_"+str(percentile)+".shp"))